<a href="https://colab.research.google.com/github/thangarani/project/blob/master/TextSummarization_Bertsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BertSum Text Summarization


---







Clone [BertSum](https://github.com/nlpyang/BertSum) project

In [ ]:
!git clone https://github.com/nlpyang/BertSum.git

In [ ]:
cd /content/BertSum/

To make it simpler and less time consuming. I Download and unziped the stories directories for both CNN and Daily Mail in my personnal pc. Put all .story files in one directory (e.g...BertSum/raw_stories) and uploaded it into my github account. You can directly clone the .story files.

In [ ]:
!git clone https://github.com/thangarani/raw_stories.git

Download and Install Stanford Core NLP

---



We will need Stanford CoreNLP to tokenize the data. Download it [here](https://stanfordnlp.github.io/CoreNLP/) and unzip it. Then add the following command:

In [ ]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-latest.zip

In [ ]:
!sudo apt install unzip

In [ ]:
!unzip stanford-corenlp-latest.zip

In [ ]:
!jar -tvf /content/BertSum/stanford-corenlp-full-2020-07-31/stanford-corenlp-4.1.0.jar

In [ ]:
%env CLASSPATH=/content/BertSum/stanford-corenlp-full-2020-07-31/stanford-corenlp-4.1.0.jar

Install pytorch_pretrained_bert, tensorboardX, multiprocess

---



In [ ]:
!pip3 install pytorch_pretrained_bert
!pip install tensorboardX
!pip install multiprocess

Pyrouge and Rouge and the corresponding supporting files

---



In [ ]:
!git clone https://github.com/andersjo/pyrouge.git

In [ ]:
cd pyrouge/tools/ROUGE-1.5.5

In [ ]:
pip install perl

In [15]:
!export ROUGE_EVAL_HOME="/content/BertSum/pyrouge/tools/ROUGE-1.5.5/data/"

In [ ]:
cd data/WordNet-2.0-Exceptions/

In [ ]:
!apt-get install synaptic

In [ ]:
!./buildExeptionDB.pl . exc WordNet-2.0.exc.db

In [ ]:
cd /content/BertSum/pyrouge/tools/ROUGE-1.5.5/data

In [20]:
rm WordNet-2.0.exc.db

In [21]:
!ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

In [ ]:
!apt-get install libxml-dom-perl

In [ ]:
!git clone https://github.com/bheinzerling/pyrouge.git

In [ ]:
cd pyrouge

In [ ]:
!python setup.py install

In [ ]:
!pyrouge_set_rouge_path /content/BertSum/pyrouge/tools/ROUGE-1.5.5/

In [ ]:
!python -m pyrouge.test

In [ ]:
cd /content/BertSum/

Pre-process the data


---



In [29]:
mkdir merged_stories_tokenized

In [ ]:
cd /content/BertSum/src/

File "/content/BertSum/src/models/data_loader.py", line 31, in __init__
    mask = 1 - (src == 0) -> change -> mask = ~ ( src == 0 )

In [ ]:
!python preprocess.py -mode tokenize -raw_path ../raw_stories -save_path ../merged_stories_tokenized -log_file ../logs/cnndm.log 

In [ ]:
!python preprocess.py -mode format_to_lines -raw_path ../merged_stories_tokenized -save_path ../json_data/cnndm -map_path ../urls -lower -log_file ../logs/cnndm.log

In [ ]:
!python preprocess.py -mode format_to_bert -raw_path ../json_data -save_path ../bert_data -oracle_mode greedy -n_cpus 4 -log_file ../logs/preprocess.log

Model Trainig using Bert RNN

In [ ]:
!python train.py -mode train -encoder rnn -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_rnn -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 5000 -save_checkpoint_steps 25000 -batch_size 3000 -decay_method noam -train_steps 50000 -accum_count 2 -log_file ../logs/bert_rnn -use_interval true -warmup_steps 10000 -rnn_size 768 -dropout 0.1

Model Training Using Bert Transformer

In [ ]:
!python train.py -mode train -encoder transformer -dropout 0.1 -bert_data_path ../bert_data/cnndm -model_path ../models/bert_transformer -lr 2e-3 -visible_gpus 0 -gpu_ranks 0 -world_size 1 -report_every 5000 -save_checkpoint_steps 25000 -batch_size 3000 -decay_method noam -train_steps 50000 -accum_count 2 -log_file ../logs/bert_transformer -use_interval true -warmup_steps 10000 -ff_size 2048 -inter_layers 2 -heads 8

-mode can be {train, validate, test}, where validate will inspect the model directory and evaluate the model for each newly saved checkpoint, test need to be used with -test_from, indicating the checkpoint you want to use

Model Evaluation

---



- Bert RNN

In [ ]:
python train.py -mode validate -bert_data_path ../bert_data/cnndm -model_path ../models/bert_rnn  -visible_gpus 0  -gpu_ranks 0 -batch_size 30000  -log_file LOG_FILE  -result_path RESULT_PATH -test_all -block_trigram true

- Bert Transformer

In [ ]:
python train.py -mode validate -bert_data_path ../bert_data/cnndm -model_path ../models/bert_transformer  -visible_gpus 0  -gpu_ranks 0 -batch_size 30000  -log_file LOG_FILE  -result_path RESULT_PATH -test_all -block_trigram true